<a href="https://colab.research.google.com/github/ailuj/BayesianDeepLearning/blob/master/First_mcdrop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Import and Preprocessing

In [1]:
import pandas as pd

from google.colab import drive
drive.mount('/content/drive',force_remount=True)
!pip install -U -q kaggle
!mkdir -p ~/.kaggle
from google.colab import files
files.upload()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"batuhanipekci","key":"5d999081520e00512450c7712eb80936"}'}

In [2]:
!cp kaggle.json ~/.kaggle/
!kaggle datasets download -d wendykan/lending-club-loan-data 
from zipfile import ZipFile
import pandas as pd
zip_file = ZipFile('lending-club-loan-data.zip')
zip_file.namelist()
loan = pd.read_csv(zip_file.open('loan.csv'))

 97% 233M/240M [00:06<00:00, 48.5MB/s]
100% 240M/240M [00:06<00:00, 41.0MB/s]


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
loan = pd.read_csv(zip_file.open('loan.csv'))

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
loan = loan[(loan['loan_status'] != 'Current')]
import numpy as np
def ifelse_pd(x, check, yes,no):
  if x in check:
    res = yes
  else:
    res = no
  return(res)


loan['binary'] = loan['loan_status'].apply(
    lambda x: ifelse_pd(x,['Charged Off', 'Default',
                          'Late (31-120 days)',
                          'Does not meet the credit policy. Status:Charged Off'],'Default', 'Not_Default'))

selected = loan[['total_pymnt', 'annual_inc', 'int_rate','revol_bal','loan_amnt','purpose','binary']]
selected = pd.get_dummies(selected)

selected = selected.dropna()
data = selected.drop(columns = ['binary_Default', 'binary_Not_Default'])
target = selected[['binary_Default', 'binary_Not_Default']]



In [5]:
selected.head()

,total_pymnt,annual_inc,int_rate,revol_bal,loan_amnt,purpose_car,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,...,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,binary_Default,binary_Not_Default
0,5861.071414,24000.0,10.65,13648.0,5000.0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1008.710000,30000.0,15.27,1687.0,2500.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,3003.653644,12252.0,15.96,2956.0,2400.0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
3,12226.302212,49200.0,13.49,5598.0,10000.0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
5,5631.377753,36000.0,7.90,7963.0,5000.0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1


In [0]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.33, random_state=42)

scaler_tr = preprocessing.StandardScaler().fit(X_train)
X_train = scaler_tr.transform(X_train)

scaler_test = preprocessing.StandardScaler().fit(X_test)
X_test = scaler_test.transform(X_test)

# MLP Architecture

In [7]:
from keras.models import Sequential
from keras.utils import np_utils
from keras.layers.core import Dense, Activation, Dropout
import time

Using TensorFlow backend.


In [0]:
dropout_prob = 0.20 # dropout should be between 0.05 and 0.5
n_epochs = 100
tau = 1.0

n_hidden = [15,15] # 2 hidden layers, each with 15 nodes
N = X_train.shape[0]
input_dim = X_train.shape[1]
output_dim = y_train.shape[1]

batch_size = 100

# If addditional regularizer
# lengthscale = 1e-2
# reg = lengthscale**2 * (1 - dropout) / (2. * N * tau)


In [0]:
from keras import Model, Input
from keras.layers import Dropout
from keras.layers import Dense

def mlp(input_dim, n_hidden, dropout_prob, train_activation, MCDropout = False):
  
  ### The code used to be like this, but Keras doesn't allow Dropout in the output layer
  
  #model = Sequential()
  #model.add(Dense(128, input_dim = X_train.shape[1], activation = train_activation))
  #model.add(Dropout(dropout_prob)) ## End of input layer
  #model.add(Dense(n_hidden[0], activation = train_activation)) ## First hidden layer 
  # We can consider adding a regularizer during regression problems
  #for i in range(len(n_hidden)-1): ## Remaining hidden layers
  #  model.add(Dropout(dropout_prob))
  #  model.add(Dense(n_hidden[i+1], activation = train_activation))
  #if MCDropout == True:
  #  model.add(Dropout(dropout_prob)) ## Adding dropout before the output layer: Trick is here!
  #model.add(Dense(output_dim, activation = 'softmax')) ## Output layer
  
  #return model

  # New model
  inputs = Input(shape=(X_train.shape[1],))
  inter = Dropout(dropout_prob)(inputs, training=True)
  inter = Dense(n_hidden[0], activation=train_activation)(inter)
  for i in range(len(n_hidden) - 1):
    inter = Dropout(dropout_prob)(inter, training=True)
    inter = Dense(n_hidden[i+1], activation=train_activation)(inter)
  if MCDropout == True:
    inter = Dropout(dropout_prob)(inter, training=True)
  outputs = Dense(y_train.shape[1],activation = 'softmax')(inter)
  model = Model(inputs, outputs)
  
  return model

 # Usual MLP

In [10]:
start_time = time.time()
model = mlp(input_dim = input_dim, n_hidden = n_hidden, 
            dropout_prob = dropout_prob, train_activation = 'relu',MCDropout = False)
model.compile(loss = 'binary_crossentropy',optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=n_epochs, verbose=1)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """


Epoch 1/100
191349/191349 [==============================] - 14s 74us/step - loss: 0.4072 - acc: 0.8159
Epoch 2/100
191349/191349 [==============================] - 12s 64us/step - loss: 0.3691 - acc: 0.8344
Epoch 3/100
191349/191349 [==============================] - 12s 64us/step - loss: 0.3518 - acc: 0.8413
Epoch 4/100
191349/191349 [==============================] - 12s 65us/step - loss: 0.3397 - acc: 0.8473
Epoch 5/100
191349/191349 [==============================] - 12s 64us/step - loss: 0.3318 - acc: 0.8511
Epoch 6/100
191349/191349 [==============================] - 12s 65us/step - loss: 0.3242 - acc: 0.8563
Epoch 7/100
191349/191349 [==============================] - 12s 65us/step - loss: 0.3197 - acc: 0.8598
Epoch 8/100
191349/191349 [==============================] - 12s 65us/step - loss: 0.3184 - acc: 0.8601
Epoch 9/100
191349/191349 [==============================] - 12s 65us/step - loss: 0.3156 - acc: 0.8617
Epoch 10/100
191349/191349 [==============================] - 13

In [11]:
pred = model.predict(X_test, verbose=1)

94247/94247 [==============================] - 3s 35us/step


In [12]:
# Accuracy

from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,pred)


0.919692893644511

# Bayesian MLP with Dropout Trick

In [13]:
model = mlp(input_dim = input_dim, n_hidden = n_hidden, 
            dropout_prob = dropout_prob, train_activation = 'relu', 
            MCDropout = True)
model.compile(loss = 'binary_crossentropy',optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=n_epochs, verbose=0)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """


In [0]:
# Run the model T times, each time we obtain a different output.
# Collect outputs. It is our distribution
# Gal argues that this is the same thing as sampling from the posterior

T = 100
        
probs = []
for _ in range(T):
    probs += [model.predict(X_test,verbose=0)]

In [16]:

predictive_mean = np.mean(probs, axis=0)
predictive_variance = np.var(probs, axis=0)
                                  
l = 100 
# I need to check out how to choose l and tau

tau = l**2 * (1 - dropout_prob) / (2 * N * 1)
predictive_variance += tau**-1




0.9658339105646352

In [19]:
# Accuracy
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,predictive_mean)


0.9658339105646352

In [20]:
probs

[array([[0.02703005, 0.9729699 ],
        [0.43370053, 0.56629944],
        [0.07842223, 0.92157775],
        ...,
        [0.80328125, 0.19671875],
        [0.7300749 , 0.26992512],
        [0.00992135, 0.9900787 ]], dtype=float32),
 array([[0.07976808, 0.92023194],
        [0.5606283 , 0.43937168],
        [0.02320495, 0.976795  ],
        ...,
        [0.11592176, 0.8840782 ],
        [0.836718  , 0.16328196],
        [0.01864277, 0.9813572 ]], dtype=float32),
 array([[0.04799726, 0.9520027 ],
        [0.8256119 , 0.17438816],
        [0.05911325, 0.9408868 ],
        ...,
        [0.54088885, 0.45911118],
        [0.82743967, 0.1725603 ],
        [0.22653343, 0.7734665 ]], dtype=float32),
 array([[0.07104732, 0.9289527 ],
        [0.5449312 , 0.45506883],
        [0.0064915 , 0.9935086 ],
        ...,
        [0.62459934, 0.3754006 ],
        [0.8478918 , 0.15210822],
        [0.01561419, 0.9843858 ]], dtype=float32),
 array([[0.04710552, 0.95289445],
        [0.66064674, 0.3393533

In [18]:
predictive_variance

array([[47.842007, 47.842007],
       [47.876347, 47.876347],
       [47.86457 , 47.86457 ],
       ...,
       [47.93034 , 47.93034 ],
       [47.868946, 47.868946],
       [47.842712, 47.842712]], dtype=float32)